In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import json
import csv
import random
from datetime import datetime, timedelta

spark = SparkSession.builder.appName("EMpresaYAnalseDeDados").getOrCreate()



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/09 16:56:11 WARN Utils: Your hostname, null, resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface wlp3s0)
25/11/09 16:56:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 16:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
def gerar_dados_vendas():
    produtos = ["Notebook","Smartphone", "Tablet", "Fone de Ouvido", "Smartwatch"]
    with open('dados_vendas.csv','w', newline='') as arquivo:
        escritor = csv.writer(arquivo)
        escritor.writerow(["id_venda", "id_produto", "produto", "quantidade","preco","data"])
        for i in range(1000):
            id_produto = random.randint(1,5)
            escritor.writerow([i, id_produto,produtos[id_produto-1],random.randint(1,5),"{:.2f}".format(random.uniform(100,1000),(datetime.now() - timedelta(days=random.randint(0,30))).strftime("%Y-%m-%d %H:%M:%S"))])

def gerar_dados_comportamento_cliente():
    acoes = ["visualizacao", "adicionar_ao_carrinho","compra", "avaliacao"]
    dados = []
    for i in range(1000):
        dados.append({
            "id_cliente": random.randint(1,100),
            "id_produto":random.randint(1,5),
            "acao":random.choice(acoes),
            "data_hora":(datetime.now() - timedelta(minutes=random.randint(0,1440))).isoformat()
        })
    with open('comportamento_cliente.json','w') as arquivo:
        json.dump(dados, arquivo)
    
    
def gerar_dados_estoque():
    produtos = ["Notebook","Smartphone", "Tablet", "Fone de Ouvido", "Smartwatch"]
    with open('dados_estoque.csv','w', newline='') as arquivo:
        escritor = csv.writer(arquivo)
        escritor.writerow(["id_produto", "nome_produto", "quantidade","ultima_atualizacao"])
        for i in range(1, 6):
            escritor.writerow([
                i,
                produtos[i-1],
                random.randint(0,1000),
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ])
    
    
    
    
#Gerar todso os dados
gerar_dados_vendas()
gerar_dados_comportamento_cliente()
gerar_dados_estoque()

In [ ]:
#Carregar dados de vendas
df_vendas = spark.read.csv("./dados_vendas.csv", header=True, inferSchema=True)

#carregar dados de comportamento do cliente
df_comportamento_cliente = spark.read.json("./comportamento_cliente.json")

#carregar dados de estoque
df_estoque = spark.read.csv("./dados_estoque.csv" header=True, inferSchema=True)

#processar dados de vendas
resumo_vendas = df_vendas.groupBy("id_produto", "produto") \
    .agg(sum("quantidade").alias("quantidade_total_vendida"),
         sum(col("quantidade") * col("preco")).alias("receita_total"))
    
#Processar dados de comportamento do cliente
resumo_comportamento_cleiente = df_comportamento_cliente.groupBy("id_produto","acao")     \
    .count() \
    .groupBy("id_produto") \
    .agg(sum(when(col("acao") == "visualizacao", col("count")).otherwise(0)).alias("visualizacao"),
         sum(when(col("acao") == "adicionar_ao_carrinho", col("count")).otherwise(0)).alias("adicionar ao carrinho"),
         sum(when(col("acao") == "compra", col("count")).otherwise(0)).alias("compras"),
         sum(when(col("acao") == "avaliacao", col("count")).otherwise(0)).alias("avaliacoes"))
        
#processar dados de estoque
produtos_baixo_estoque = df_estoque.filter(col("quantidade") < 100)

#Juntar dados de vendas e estoque
vendas_estoque = resumo_vendas.join(df_estoque, resumo_vendas.id_produto == df_estoque.id_produto)


#Calcular produtos mais vendidos om baixo estoque

mais_vendidos_baixo_estoque = vendas_estoque.filter(col("quantidade") < 100) \
    .orderBy(col("Quantidade_total_vendida").desc()) \
    .limit(5)
    
#Analisar comportamento do cliente para produtos mais vendidos
produtos_top = resumo_vendas.orderBy(col("quantidade_total_vendida").desc()).limit(5)
comportamento_produtos_top = resumo_comportamento_cleiente.join(produtos_top, resumo_comportamento_cleiente.id_produto)


#Calcular taxa de conversão

taxa_conversao = resumo_comportamento_cleiente.withColumn("taxa_conversao", col("compras")/col("visulizacoes") + 0.1)# adicionar 0.1 para evitar dividir por zero

print("Top 5 produtos mais vendidos com baixo estoque:")
mais_vendidos_baixo_estoque.show()

print("\nxomportamento do cliente para os produtos mais vendidos:")
comportamento_produtos_top.show()

print("\nTaxa de conversão por produto:")
taxa_conversao.orderBy(col("taxa_conversao").desc()).show()

#Salvar resultado em arquivo csv
'''
mais_vendidos_baixo_estque.write.csv("mais_vendidos_baixo", header=True)
comportamentp_produtos_top.write.csv("comportamento_produtos_top.csv, header=True")
taxa_conversao.write.csv("taxa_conversao.csv", header=True)

'''



